In [1]:
from llama_cpp.llama import LlamaGrammar

HARRY_POTTER_GBNF = r"""
ws ::= ([ \t\n] ws)?

string ::=
  "\"" (
    [^"\\\x7F\x00-\x1F] |
     "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])
  )* "\""


digit ::= "0" | "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9"

one-or-two-digits ::= digit | digit digit

zero-to-two-digits ::= "" | digit | digit digit

house ::= "\"Gryffindor\"" | "\"Hufflepuff\"" | "\"Ravenclaw\"" | "\"Slytherin\""

blood ::= "\"Muggle-born\"" | "\"Half-blood\"" | "\"Pure-blood\""

wand ::= (
  "{\n" ws
    "\"wood\": " string "," ws
    "\"core\": " string "," ws
     "\"length\": " one-or-two-digits "." zero-to-two-digits ws
  "}"
)

character ::= (
  "{\n" ws
    "\"name\": " string "," ws
    "\"house\": " house "," ws
    "\"blood status\": " blood "," ws
    "\"wand\": " wand ws
  "}"
)

root ::= "[\n  " character (",\n" ws character)* ws "]"
"""

grammar = LlamaGrammar.from_string(HARRY_POTTER_GBNF, verbose=False)


In [2]:
from llama_cpp.llama import Llama

llm = Llama(
    model_path="/Users/aidan/models/Meta-Llama-3-8B-Instruct.Q4_0.gguf",
    n_gpu_layers=-1,
    verbose=False,
)

output = llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": "Using JSON, describe these Harry Potter characters: "
            + "Ron Weasley, Snape.",
        },
    ],
    grammar=grammar,
)
print(output["choices"][0]["message"]["content"])
# >>> [
# >>>   {
# >>>     "name": "Ronald Bilius Weasley",
# >>>     "house": "Gryffindor",
# >>>     "blood status": "Pure-blood",
# >>>     "wand": {
# >>>       "wood": "Holly",
# >>>       "core": "Veela hair",
# >>>       "length": 10.5
# >>>     }
# >>>   },
# >>>   {
# >>>     "name": "Severus Tobias Snape",
# >>>     "house": "Slytherin",
# >>>     "blood status": "Half-blood",
# >>>     "wand": {
# >>>       "wood": "Blackthorn",
# >>>       "core": "Unicorn hair",
# >>>       "length": 11.75
# >>>     }
# >>>   }
# >>> ]

[
  {
    "name": "Ronald Bilius Weasley",
    "house": "Gryffindor",
    "blood status": "Pure-blood",
    "wand": {
      "wood": "Holly",
      "core": "Veela hair",
      "length": 10.5
    }
  },
  {
    "name": "Severus Tobias Snape",
    "house": "Slytherin",
    "blood status": "Half-blood",
    "wand": {
      "wood": "Blackthorn",
      "core": "Unicorn hair",
      "length": 11.75
    }
  }
]
